<a href="https://colab.research.google.com/github/Kira1108/PromptEngineering/blob/main/PromptEngineering_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
!pip install openai parse
clear_output()

In [ ]:
import openai
openai.api_key = ""

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# I. Write Clear and Specific Instructions

## 1. Use Delimeters

使用delimeters, 把Prompt的内容和Text的内容分开, delimeters可以是各种标点符号，xml标签
- ``` triple backtick
- """ triple quotes
- < > angle brackets
- `<tag>` `</tag>` tag


你的目的是要告诉GPT，这是一个独立的section

In [ ]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

To guide a model towards the desired output and reduce the chances of irrelevant or incorrect responses, it is important to provide clear and specific instructions, which may be longer prompts that provide more clarity and context for the model.


## 2. Ask for structured output

要的回json，才能免的后面去正则

```
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
```

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret of the Haunted Mansion",
    "author": "Lila Rose",
    "genre": "Mystery"
  }
]


In [ ]:
import json
import pprint
data = json.loads(response)
pprint.pp(data)

[{'book_id': 1,
  'title': 'The Lost City of Zorath',
  'author': 'Aria Blackwood',
  'genre': 'Fantasy'},
 {'book_id': 2,
  'title': 'The Last Survivors',
  'author': 'Ethan Stone',
  'genre': 'Science Fiction'},
 {'book_id': 3,
  'title': 'The Secret of the Haunted Mansion',
  'author': 'Lila Rose',
  'genre': 'Mystery'}]


## 3. Check Conditions
```
if condition safisfied:
    do something
    return result
else:
    return exception result
```

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!


## 4. Practice

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in JSON format, a list containing steps,
each step contain keys `step_order` and `step_content`

If the text does not contain a sequence of instructions, \ 
then simply response a JSON with one key 'message' and value 'No steps provided.'\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
json.loads(response)

[{'step_order': 1,
  'step_content': 'First, you need to get some water boiling.'},
 {'step_order': 2,
  'step_content': "While that's happening, grab a cup and put a tea bag in it."},
 {'step_order': 3,
  'step_content': 'Once the water is hot enough, just pour it over the tea bag.'},
 {'step_order': 4,
  'step_content': 'Let it sit for a bit so the tea can steep.'},
 {'step_order': 5,
  'step_content': 'After a few minutes, take out the tea bag.'},
 {'step_order': 6,
  'step_content': 'If you like, you can add some sugar or milk to taste.'},
 {'step_order': 7,
  'step_content': "And that's it! You've got yourself a delicious cup of tea to enjoy."}]

拿着这个东西，你就能去渲染页面去了，多骚

In [ ]:
text_1 = f"""
The weather is good today.
We are going to a restaurant.
That's happy
"""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in JSON format, a list containing steps,
each step contain keys `step_order` and `step_content`

If the text does not contain a sequence of instructions, \ 
then simply response a JSON with one key 'message' and value 'No steps provided.'\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
json.loads(response)

{'message': 'No steps provided.'}

## 5. Few Shot prompt

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends with the wind but never breaks. It is the ability to bounce back from adversity and keep moving forward, even when things get tough. Just like a tree that grows stronger with each storm it weathers, resilience is a quality that can be developed and strengthened over time.


# II. Give the model time to think

## 1. Chain of Thought

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into Chinese.
3 - List each name in the Chinese summary.
4 - Output a json object that contains the following \
keys: chinese_summary with string value, num_names with int value, names with list of strings.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Siblings Jack and Jill go on a quest to fetch water from a hilltop well, but misfortune strikes as they both tumble down the hill, yet they remain undeterred and continue exploring with delight.

2 - 兄妹杰克和吉尔前往山顶井取水，但不幸的是他们都滚下了山，尽管如此他们仍然保持着冒险精神并继续愉快地探索。

3 - 杰克，吉尔

4 - 
{
  "chinese_summary": "兄妹杰克和吉尔前往山顶井取水，但不幸的是他们都滚下了山，尽管如此他们仍然保持着冒险精神并继续愉快地探索。",
  "num_names": 2,
  "names": ["杰克", "吉尔"]
}


任务完成的挺牛逼，但是结果没法解析

In [ ]:
response.split("\n")

['1 - Siblings Jack and Jill go on a quest to fetch water from a hilltop well, but misfortune strikes as they both tumble down the hill, yet they remain undeterred and continue exploring with delight.',
 '',
 '2 - 兄妹杰克和吉尔前往山顶井取水，但不幸的是他们都滚下了山，尽管如此他们仍然保持着冒险精神并继续愉快地探索。',
 '',
 '3 - 杰克，吉尔',
 '',
 '4 - ',
 '{',
 '  "chinese_summary": "兄妹杰克和吉尔前往山顶井取水，但不幸的是他们都滚下了山，尽管如此他们仍然保持着冒险精神并继续愉快地探索。",',
 '  "num_names": 2,',
 '  "names": ["杰克", "吉尔"]',
 '}']

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into Chinese.
3 - List each name in the Chinese summary.
4 - Output a json object that contains the 
  following keys: chinese_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Chinese summary>
Output JSON: <json with summary ,num_names, and names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill go on a quest to fetch water, but misfortune strikes and they tumble down a hill, returning home slightly battered but with undimmed adventurous spirits. 
Translation: Jack 和 Jill 去取水，但不幸的是他们跌倒了，回家时有些受伤，但他们的冒险精神仍然不减，继续愉快地探索。
Names: Jack, Jill
Output JSON: {"chinese_summary": "Jack 和 Jill 去取水，但不幸的是他们跌倒了，回家时有些受伤，但他们的冒险精神仍然不减，继续愉快地探索。", "num_names": 2, "names": ["Jack", "Jill"]}


In [ ]:
from parse import parse
r = parse("Output JSON: {json_data}",response.split("\n")[-1])
json.loads(r.named['json_data'])

{'chinese_summary': 'Jack 和 Jill 去取水，但不幸的是他们跌倒了，回家时有些受伤，但他们的冒险精神仍然不减，继续愉快地探索。',
 'num_names': 2,
 'names': ['Jack', 'Jill']}

想尽一切办法最后取到了Json    
- 1. 细化Chain of Thought
- 2. 关键部分由Json给出

## 2. Workout Before Conclusion   

比较巧妙的利用了一种补全的方式， Chain of Thought了

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect
